## Check pCVP2-BRI1-mCitrine specificity in scRNA-seq

Here we perform analysis for new and published samples against the custom reference. The results are exported and inlcuded in the SI table. 

In [1]:
library(tidyverse)
library(Seurat)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(GeneOverlap)
library(gprofiler2)
library(ggrepel)
library(ggplot2)
library(muscat)
library(purrr)
library(limma)
library(scran)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.1.8
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Attaching SeuratObject


Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualizati

In [2]:
library(future)
#for 200gb ram 
options(future.globals.maxSize = 200000 * 1024^2)

In [3]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: AlmaLinux 9.3 (Shamrock Pampas Cat)

Matrix products: default
BLAS/LAPACK: /hpc/group/pbenfeylab/tmn23/miniconda3/envs/muscat/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] future_1.31.0               scran_1.26.0               
 [3] scuttle_1.8.0               SingleCellExperiment_1.20.0
 [5] SummarizedExperiment_1.28.0 Biobase_2.58.0             
 [7] GenomicRanges_1.50.0        GenomeInfoDb_1.34.8        
 [9]

In [ ]:
rc.integrated <- readRDS("../../CheWei/scRNA-seq/Integrated_Objects/rc.integrated_8S_CVP_BRI1_seu3_annotated_20230316.rds")

In [ ]:
rc.integrated

In [ ]:
table(rc.integrated$orig.ident, rc.integrated$geno)

In [ ]:
table(rc.integrated$orig.ident, rc.integrated$source)

In [ ]:
feature_names <- read_tsv("../data/features.tsv.gz", col_names = c("AGI", "Name", "Type")) %>%
  select(-Type) %>%
  distinct()

In [ ]:
rc.integrated$geno <- factor(rc.integrated$geno, levels=c("WT", "bri1_T", "pCVP2_BRI1_Citrine_bri1_T"))

## Cell and developmental stage metadata

- Developmental stage: `time_zone`
- Cell type:`cell_type`
- Combination of cell type and developmental stage: `time_zone_cell_type`

In [ ]:
order <- c("Quiescent Center", "Ground Tissue","Columella", "Lateral Root Cap", "Atrichoblast", "Trichoblast", "Cortex", "Endodermis", "Phloem","Protophloem", "Xylem", "Procambium","Pericycle","Phloem Pole Pericycle", "Protoxylem", "Metaxylem", "Unknown")
palette <- c("#9400D3", "#DCD0FF","#5AB953", "#BFEF45", "#008080", "#21B6A8", "#82B6FF", "#0000FF","#E6194B", "#DD77EC", "#9A6324", "#FFE119", "#FF9900", "#FFD4E3", "#9A6324", "#DDAA6F", "#EEEEEE")
rc.integrated$cell_type <- factor(rc.integrated$cell_type, levels = order[sort(match(unique(rc.integrated$cell_type),order))])
color <- palette[sort(match(unique(rc.integrated$cell_type),order))]

In [ ]:
options(repr.plot.width=16.5, repr.plot.height=6)
(Celltype_umap <- DimPlot(rc.integrated, 
                      reduction = "umap", 
                      group.by = "cell_type", 
                      cols = color, split.by = 'geno', 
                      ncol=3, 
                      pt.size = 0.5))

ggsave("../output/CVP/Cell_type_umap_square_all_samples.pdf", width=16.5, height=6)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)

DefaultAssay(rc.integrated) <- "SCT"
# expression of the transgene
FeaturePlot(rc.integrated, features="BRI1-mCitrine", split.by = "geno", order=T, max.cutoff = "q90", pt.size = 0.5)

ggsave("../output/CVP/BRI1-mCitrine_expression_all_samples.pdf", width=18.3, height=6)

In [ ]:
# expression of CVP2
options(repr.plot.width = 18, repr.plot.height = 6)

DefaultAssay(rc.integrated) <- "SCT"
# expression of the transgene
FeaturePlot(rc.integrated, features="AT1G05470", split.by = "geno", order=T, max.cutoff = "q90", pt.size = 0.5)

ggsave("../output/CVP/CVP2_AT1G05470_expression_all_samples.pdf", width=18.3, height=6)

In [ ]:
options(repr.plot.width=30, repr.plot.height=7)

DimPlot(rc.integrated, reduction = "umap", group.by = "cell_type", cols = color, split.by = 'orig.ident', pt.size = 0.75, ncol=8)

In [ ]:
DefaultAssay(rc.integrated) <- "SCT"
# expression of the transgene
FeaturePlot(rc.integrated, features="BRI1-mCitrine", split.by = "orig.ident", order=T, max.cutoff = "q80", pt.size = 0.5)

In [ ]:
DefaultAssay(rc.integrated) <- "SCT"
# expression of the transgene
FeaturePlot(rc.integrated, features="AT1G05470", split.by = "orig.ident", order=T, max.cutoff = "q80", pt.size = 0.5)

In [ ]:
table(rc.integrated$orig.ident, rc.integrated$cell_type)

In [ ]:
table(rc.integrated$orig.ident, rc.integrated$time_zone)

## Convert to sce

In [ ]:
#  construct sce manually
my_metadata <- data.frame(sample_id = rc.integrated$orig.ident,
                              group_id = rc.integrated$geno,
                              cluster_id = rc.integrated$cell_type, 
                             date=rc.integrated$rep) # include experimental rep as co-variate

sce <- SingleCellExperiment(assays = list(counts = rc.integrated@assays$RNA@counts),
	                            colData = my_metadata)

In [ ]:
    (sce <- prepSCE(sce, 
        kid = "cluster_id", # subpopulation assignments
        gid = "group_id",   # group IDs (ctrl/stim)
        sid = "sample_id",    # sample IDs (ctrl/stim.1234)
        drop = FALSE))        # drop all other colData columns

## pre-filtering

In [ ]:
# remove undetected genes
sce <- sce[rowSums(counts(sce) > 0) > 0, ]
dim(sce)

In [ ]:
# create pseudobulk profiles
pb <- aggregateData(sce,
    assay = "counts", fun = "sum",
    by = c("cluster_id", "sample_id"))
# one sheet per subpopulation
assayNames(pb)

In [ ]:
# pseudobulks for 1st subpopulation
t(head(assay(pb)))

In [ ]:
# experiment info for contrasts, add dates from csv

ei <- metadata(sce)$experiment_info

ei

ei$rep <- c(1, 1, 1, 2, 2, 2, 3, 3)
ei
#sample_date <- select(bscs, sample_id=sample, date=rep)

#ei <- left_join(ei, sample_date)

#ei

In [ ]:
mm <- model.matrix(~ 0 + ei$group_id + ei$rep)
dimnames(mm) <- list(ei$sample_id, c(levels(ei$group_id), "rep"))

mm

In [ ]:
contrast <- makeContrasts("pCVP2_BRI1_Citrine_bri1_T-bri1_T", levels = mm)

contrast

In [ ]:
res <- pbDS(pb, design = mm, 
            contrast = contrast, 
            method="edgeR", 
            min_cells=5, 
            filter = c("none"))

### DEG results

In [ ]:
# DEG results with gene freqs
(res_to_write_frq <- resDS(sce, res, bind = "row", cpm=TRUE, frq=T))

In [ ]:
## all genes as background

all_bg <- res_to_write_frq

In [ ]:
all_bg %>% filter(gene=="BRI1-mCitrine") %>%
arrange(desc(briTR.cpm))

In [ ]:
length(unique(all_bg$gene))

In [ ]:
#total DE genes p_adj.loc < 0.05, abs(logFC) > 1.5
sig_DE <- filter(res_to_write_frq, p_adj.loc<=0.05 & abs(logFC) > log2(1.5))
sig_DE <- left_join(sig_DE, feature_names, by=c("gene"="AGI"))

length(unique(sig_DE$gene))

In [ ]:
# filter gene freqs to avoid calling lowly detected genes
sig_DE_fil <- filter(sig_DE, WT.frq >=0.1 | bri1_T.frq >=0.1 | pCVP2_BRI1_Citrine_bri1_T.frq>=0.1)

In [ ]:
length(unique(sig_DE_fil$gene))

In [ ]:
# load TFs
TF_list <- read_csv("../data/Kay_TF_thalemine_annotations.csv", col_names = c("gene", "TF_Name", "Description")) 

In [ ]:
sig_DE_fil <- left_join(sig_DE_fil, TF_list)

In [ ]:
# label up vs down
sig_DE_fil <- sig_DE_fil %>%
  mutate(up_dn_label = case_when(logFC >=log2(1.5) ~ "Up",  
                                       logFC <=log2(1/1.5) ~ "Down",
                                       TRUE ~ "Not DE"))

sig_DE_fil$clust_up_dn <- paste(sig_DE_fil$cluster_id, sig_DE_fil$up_dn_label, sep="_")

sig_DE_fil

In [ ]:
sig_DE_fil
write.csv(sig_DE_fil, file = "../output/CVP/pCVP2_Citrine_celltype_EdgeR_q0.05_FC1.5_r_v_4_20230317.csv")

In [ ]:
# add DE and up/dn to total list
sig_to_join <- sig_DE_fil %>%
mutate(clust_gene=paste(cluster_id, gene, sep="_")) %>%
select(clust_gene, up_dn_label, clust_up_dn)

In [ ]:
# join all genes list to DE labels
all_bg <- mutate(all_bg, clust_gene=paste(cluster_id, gene, sep="_"))

all_bg <- left_join(all_bg, feature_names, by=c("gene"="AGI"))

all_bg$DE <- all_bg$clust_gene %in% sig_to_join$clust_gene


all_bg <- all_bg %>%
left_join(sig_to_join, by="clust_gene") %>%
arrange(all_bg, p_adj.loc)

write.csv(all_bg, file = "../output/CVP/all_genes_pCVP2_Citrine_celltype_EdgeR_q0.05_FC1.5_r_v_4_20230317.csv")

# Plotting

We focus on the Benfey lab samples that were collected side-by-side for plotting

In [ ]:
integrated.de <- subset(rc.integrated, 
                        subset = orig.ident %in% c("sc_130",
                                              "sc_131",
                                              "sc_132",
                                              "sc_134",
                                              "sc_135",
                                              "sc_136"))

In [ ]:
integrated.de$geno <- factor(integrated.de$geno, 
                             levels=c("WT", "bri1_T", "pCVP2_BRI1_Citrine_bri1_T"))

In [ ]:
options(repr.plot.width=16.5, repr.plot.height=6)
(Celltype_umap <- DimPlot(integrated.de, 
                      reduction = "umap", 
                      group.by = "cell_type", 
                      cols = color, split.by = 'genotype', 
                      ncol=3, 
                      pt.size = 0.5))

ggsave("../output/CVP/Cell_type_umap_square.pdf", width=16.5, height=6)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)

DefaultAssay(integrated.de) <- "SCT"
# expression of the transgene
FeaturePlot(integrated.de, features="BRI1-mCitrine", split.by = "genotype", order=T, max.cutoff = "q90", pt.size = 0.5)

ggsave("../output/CVP/BRI1-mCitrine_expression.pdf", width=18.3, height=6)

In [ ]:
# expression of CVP2
options(repr.plot.width = 18, repr.plot.height = 6)

DefaultAssay(integrated.de) <- "SCT"
# expression of the transgene
FeaturePlot(integrated.de, features="AT1G05470", split.by = "genotype", order=T, max.cutoff = "q90", pt.size = 0.5)

ggsave("../output/CVP/CVP2_AT1G05470_expression.pdf", width=18.3, height=6)